In [1]:
import tkinter as tk
import math


class GraphDraw:
    def __init__(
        self,
        width=800,
        height=600,
        x_min=-10,
        x_max=10,
        functions={"None": None},
    ):
        self.functions = functions
        self.width = width
        self.height = height
        self.x_min = x_min
        self.x_max = x_max
        self.root = tk.Tk()

    def start(self):
        self.var = tk.StringVar(value=list(self.functions.keys())[0])
        option_menu = tk.OptionMenu(self.root, self.var, *self.functions.keys())
        draw_button = tk.Button(
            self.root, text="Рисовать", command=self.draw_selected_graph
        )

        self.root.title("График")
        self.root.geometry(f"{self.width}x{self.height}")

        option_menu.pack(side=tk.TOP, padx=10)
        draw_button.pack(side=tk.TOP, padx=10)

        self.canvas = tk.Canvas(self.root, width=self.width, height=self.height)
        self.canvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        self.canvas.bind(
            "<Configure>",
            lambda event: self.draw_selected_graph(),
        )

        self.root.mainloop()

    def get_y_range_with_offset(self, func, x_min, x_max, step=0.1):
        y_min = float("inf")
        y_max = float("-inf")

        x = x_min
        while x <= x_max:
            try:
                y = func(x)
                if not math.isfinite(y):  
                    x += step
                    continue
                if y < y_min:
                    y_min = y
                if y > y_max:
                    y_max = y
            except (ValueError, ZeroDivisionError):
                pass
            x += step

        max_abs = max(abs(y_min), abs(y_max))
        return max_abs * 1.2 if max_abs > 0 else 1.0

    def draw_selected_graph(self):
        selected_function_name = self.var.get()
        func = self.functions.get(selected_function_name)
        if func:
            self.canvas.delete("all")
            y_range = self.get_y_range_with_offset(func, self.x_min, self.x_max)
            self.draw_graph(func, self.x_min, self.x_max, -y_range, y_range)

    def draw_graph(self, func, x_min, x_max, y_min, y_max):
        window_width = self.canvas.winfo_width()
        window_height = self.canvas.winfo_height()
        
        x_scale = window_width / (x_max - x_min)
        y_scale = window_height / (y_max - y_min)

        self.canvas.create_line(
            0,
            window_height / 2,
            window_width,
            window_height / 2,
            fill="black",
            arrow=tk.LAST,
        )
        self.canvas.create_line(
            window_width / 2,
            0,
            window_width / 2,
            window_height,
            fill="black",
            arrow=tk.FIRST,
        )

        self.canvas.create_text(window_width - 10, window_height / 2 - 10, text="X", fill="black")
        self.canvas.create_text(window_width / 2 + 10, 10, text="Y", fill="black")

        points_to_draw = []
        
        for x_pixel in range(window_width):
            x = x_min + x_pixel / x_scale
            try:
                y = func(x)
                if math.isfinite(y): 
                    y_pixel = window_height - (y - y_min) * y_scale
                    points_to_draw.append((x_pixel, y_pixel))
                else:
                    points_to_draw.append(None)  
            except (ValueError, ZeroDivisionError):
                points_to_draw.append(None)

        for i in range(1, len(points_to_draw)):
            if points_to_draw[i-1] is not None and points_to_draw[i] is not None:
                self.canvas.create_line(
                    points_to_draw[i-1][0], points_to_draw[i-1][1],
                    points_to_draw[i][0], points_to_draw[i][1],
                    fill="blue", width=2
                )

functions = {
    "sin(x)": lambda x: math.sin(x),
    "x^0.5": lambda x: math.sqrt(x) ,
    "1/0": lambda x: 1/0,
    "tg(x)": lambda x: math.tan(x),
    "x^2": lambda x: x**2,
    "x^3": lambda x: x**3,
    "x": lambda x: x,
}

graph_draw = GraphDraw(
    functions=functions,
)
graph_draw.start()